In [1]:
import numpy as np
import glob
import os

# 1. load dataset

In [2]:
def load_graph_data(file_path:str):
    
    with open(file_path , 'r') as f:
        data = f.read().strip()

    data = [  i.strip().split(',') for i in  data.split('\n')]
    return data

In [3]:
def load_IBmdata(file_path:str):
    
    with open(file_path , 'r') as f:
        data = f.read().strip()
        
    data = [  i.split()[-2:] for i in  data.split('\n')]
    return data

# 2. graph class

In [4]:
class Graph():
    def __init__(self , graph_data:list ) -> None:

        # init auth and hub matrix , index
        self.init_graph(graph_data)

    def init_graph(self , graph_data:list)->None:

        self.uni_nodes = set( [int(item) for sublist in graph_data for item in sublist])
        self.uni_nodes = sorted(list(self.uni_nodes) , key=lambda x:int(x) )
        self.n = len(self.uni_nodes)

        # create idx in matrix for every node
        self.node_to_idx = {  int(n):int(idx) for idx,n in  enumerate(self.uni_nodes)  }
        self.idx_to_node = {  idx:n for n,idx in self.node_to_idx.items() }

        self.hub_matrix = np.zeros([self.n,self.n]) 

        # if a->b,record as 1
        for a_node , b_node in graph_data:
            a_idx = self.node_to_idx[int(a_node)]
            b_idx = self.node_to_idx[int(b_node)]
            self.hub_matrix[a_idx][b_idx] = 1 

        self.auth_matrix = np.transpose(self.hub_matrix)
        
        self.out_neighbors = { i:np.nonzero(arr)[0] for i,arr in enumerate(self.hub_matrix)  }
        self.in_neighbors = { i:np.nonzero(arr)[0] for i,arr in enumerate(self.auth_matrix)  }
    
    def hits_algorithm(self , iter):
        
        hub_score = np.ones(self.n)
        auth_score = np.ones(self.n)

        for _ in range(iter):
            # new hub and auth score
            new_auth_score =  np.zeros( self.n )
            new_hub_score =  np.zeros( self.n )
            # auth update and hub update
            for i in range(self.n):
                new_auth_score[i]  = np.sum( hub_score[ self.in_neighbors[i] ] )
                new_hub_score[i] = np.sum( auth_score[ self.out_neighbors[i] ] )
            # norm and update
            auth_score =  new_auth_score / new_auth_score.sum()
            hub_score = new_hub_score / new_hub_score.sum()

        return hub_score , auth_score
    
    def pagerank_algorithm(self,iter , d):
        
        page_rank = np.ones( self.n )

        for _ in range(iter):
            new_page_rank = np.zeros_like(page_rank)
            for i in range(self.n):
                # for every node points to me , update page_rank score as sum of (old_page[ni]) / (ni out links)
                for n in self.in_neighbors[i]:
                    new_page_rank[i] += page_rank[n] / len(self.out_neighbors[n])
                
            page_rank = (1-d) * new_page_rank + d/self.n
            
        # norm
        page_rank = page_rank / (page_rank.sum())

        return page_rank

    def simrank_algorithm(self,iter,C):
        
        simrank_matrix = np.eye( self.n )

        for _ in range(iter):
            # create new simrank matrix
            new_simrank_matrix = np.eye( self.n )
            # update every node of a:b
            for node_a in range(self.n):
                for node_b in range(self.n):
                    if len(self.in_neighbors[node_a]) == 0 or len(self.in_neighbors[node_b]) == 0 or node_a == node_b: 
                        continue  
                    new_simrank_matrix[node_a][node_b] = self.get_simrank_score( node_a,node_b,simrank_matrix,C=C )

            simrank_matrix = new_simrank_matrix.copy()
            
        return simrank_matrix
        
    def get_simrank_score( self,node_a,node_b,old_simrank_matrix , C)->float:

        simrank_sum = 0.0
        for a in self.in_neighbors[node_a]:
            for b in self.in_neighbors[node_b]:
                simrank_sum += old_simrank_matrix[a][b]

        simrank_sum = (simrank_sum * C) / (len( self.in_neighbors[node_a])*len(self.in_neighbors[node_b])) 
        return simrank_sum

# 3. Page rank ,Hits ,SimRank
+ for every graph and IMB data,output result to 4 file and show auth + hub to stdout
+ damping factor d = 0.15
+ dacay factor C = 0.9
+ iter = 100

In [5]:
# if u wanna recreate file ,run this to clear old file
if not os.path.isdir("./result"):
    os.mkdir("./result/")
#os.system("rm ./result/*")

In [6]:
data_path = list(sorted(glob.glob("./dataset/*.txt")))
ITER = 100
DECAY_FACTOR_C = 0.9
DAMPING_FACTOR_D = 0.15
print(data_path)

['./dataset/graph_1.txt', './dataset/graph_2.txt', './dataset/graph_3.txt', './dataset/graph_4.txt', './dataset/graph_5.txt', './dataset/graph_6.txt', './dataset/ibm-5000.txt']


In [7]:
# for every dataset,print result and save to file
for graph_idx ,d_path in enumerate(data_path):

    # load data from file,two format
    if d_path.find("ibm") !=-1:
        graph_data = load_IBmdata(str(d_path))
    else:
        graph_data = load_graph_data(str(d_path))

    # create graph
    graph = None
    graph= Graph(graph_data)

    # output hit result
    hitshub,hits_auth =  graph.hits_algorithm(iter=ITER)
    pagerank = graph.pagerank_algorithm(iter=ITER,d= DAMPING_FACTOR_D)

    # print result
    print(f"Dataset [{d_path}]\n")
    print(f"Authority:\n{hits_auth}")
    print(f"Hub:\n{hitshub}")
    print(f"PageRank:\n{pagerank}")

    # output folder
    out_path = d_path.replace('dataset',"result")

    # save file
    np.savetxt( out_path.replace(".txt","_HITS_authority.txt") , hits_auth ,fmt='%.5f', newline=' ')
    np.savetxt( out_path.replace(".txt","_HITS_hub.txt") , hitshub,fmt='%.5f', newline=' ')
    np.savetxt( out_path.replace(".txt","_PageRank.txt") , pagerank,fmt='%.5f', newline=' ')

    # we only need to handle first 5 graph in simrank task
    #if graph_idx !=5:
    simrank = graph.simrank_algorithm(iter=ITER,C = DECAY_FACTOR_C)
    print(f"SimRank:\n{simrank}")
    np.savetxt( out_path.replace(".txt","_SimRank.txt") , simrank,fmt='%.5f', newline='\n')

    print("\n================================\n")


Dataset [./dataset/graph_1.txt]

Authority:
[0.  0.2 0.2 0.2 0.2 0.2]
Hub:
[0.2 0.2 0.2 0.2 0.2 0. ]
PageRank:
[0.06071611 0.11232481 0.1561922  0.19347948 0.22517367 0.25211373]
SimRank:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


Dataset [./dataset/graph_2.txt]

Authority:
[0.2 0.2 0.2 0.2 0.2]
Hub:
[0.2 0.2 0.2 0.2 0.2]
PageRank:
[0.2 0.2 0.2 0.2 0.2]
SimRank:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


Dataset [./dataset/graph_3.txt]

Authority:
[0.19098301 0.30901699 0.30901699 0.19098301]
Hub:
[0.19098301 0.30901699 0.30901699 0.19098301]
PageRank:
[0.17543859 0.32456141 0.32456141 0.17543859]
SimRank:
[[1.         0.         0.81818182 0.        ]
 [0.         1.         0.         0.81818182]
 [0.81818182 0.         1.         0.        ]
 [0.         0.81818182 0.         1.        ]]


Dataset [./dataset/graph_4.txt]

Authority:
[0.13948389 0.1

# 4. Find a way

## Q1 : add 5,1 4,1

In [8]:
graph_data = load_graph_data(data_path[0])
graph = Graph(graph_data)
hub,auth = graph.hits_algorithm(ITER)
pagerank = graph.pagerank_algorithm(iter=ITER,d= DAMPING_FACTOR_D)
print( f"Before:\n\tHub {np.round(hub,3)} Auth {np.round(auth,3)} \n\tPageRank {np.round(pagerank,3)}"  )

graph_data.append(['5','1'])
graph_data.append(['4','1'])
new_graph = Graph(graph_data)
new_hub,new_auth = new_graph.hits_algorithm(ITER)
new_pagerank = new_graph.pagerank_algorithm(iter=ITER,d= DAMPING_FACTOR_D)
print( f"After:\n\tHub {np.round(new_hub,3)} Auth {np.round(new_auth,3)} \n\tPageRank {np.round(new_pagerank,3)}"  )


Before:
	Hub [0.2 0.2 0.2 0.2 0.2 0. ] Auth [0.  0.2 0.2 0.2 0.2 0.2] 
	PageRank [0.061 0.112 0.156 0.193 0.225 0.252]
After:
	Hub [0.  0.  0.  0.5 0.5 0. ] Auth [0.5  0.   0.   0.   0.25 0.25] 
	PageRank [0.181 0.191 0.201 0.209 0.127 0.092]


## Q2

In [9]:
graph_data = load_graph_data(data_path[1])
graph = Graph(graph_data)
hub,auth = graph.hits_algorithm(ITER)
pagerank = graph.pagerank_algorithm(iter=ITER,d= DAMPING_FACTOR_D)
print( f"Before:\n\tHub {np.round(hub,3)} Auth {np.round(auth,3)} \n\tPageRank {np.round(pagerank,3)}"  )

graph_data.append(['3','1'])
graph_data.append(['3','5'])
graph_data.append(['3','2'])
new_graph = Graph(graph_data)
new_hub,new_auth = new_graph.hits_algorithm(ITER)
new_pagerank = new_graph.pagerank_algorithm(iter=ITER,d= DAMPING_FACTOR_D)
print( f"After:\n\tHub {np.round(new_hub,3)} Auth {np.round(new_auth,3)} \n\tPageRank {np.round(new_pagerank,3)}"  )

Before:
	Hub [0.2 0.2 0.2 0.2 0.2] Auth [0.2 0.2 0.2 0.2 0.2] 
	PageRank [0.2 0.2 0.2 0.2 0.2]
After:
	Hub [0.147 0.    0.558 0.147 0.147] Auth [0.264 0.264 0.    0.209 0.264] 
	PageRank [0.22  0.273 0.262 0.086 0.159]


# Q3

In [10]:
graph_data = load_graph_data(data_path[2])
graph = Graph(graph_data)
hub,auth = graph.hits_algorithm(ITER)
pagerank = graph.pagerank_algorithm(iter=ITER,d= DAMPING_FACTOR_D)
print( f"Before:\n\tHub {np.round(hub,3)} Auth {np.round(auth,3)} \n\tPageRank {np.round(pagerank,3)}"  )

graph_data.append(['3','1'])
graph_data.append(['4','1'])
new_graph = Graph(graph_data)
new_hub,new_auth = new_graph.hits_algorithm(ITER)
new_pagerank = new_graph.pagerank_algorithm(iter=ITER,d= DAMPING_FACTOR_D)
print( f"After:\n\tHub {np.round(new_hub,3)} Auth {np.round(new_auth,3)} \n\tPageRank {np.round(new_pagerank,3)}"  )


Before:
	Hub [0.191 0.309 0.309 0.191] Auth [0.191 0.309 0.309 0.191] 
	PageRank [0.175 0.325 0.325 0.175]
After:
	Hub [0.079 0.298 0.324 0.298] Auth [0.41  0.18  0.266 0.144] 
	PageRank [0.301 0.36  0.235 0.104]


# 5.Effectiveness analysis

In [11]:
import matplotlib.pyplot as plt
import time

In [12]:
data_path = list(sorted(glob.glob("./dataset/*.txt")))
ITER = 100
DECAY_FACTOR_C = 0.9
DAMPING_FACTOR_D = 0.15
print(data_path)

['./dataset/graph_1.txt', './dataset/graph_2.txt', './dataset/graph_3.txt', './dataset/graph_4.txt', './dataset/graph_5.txt', './dataset/graph_6.txt', './dataset/ibm-5000.txt']


In [13]:
record = {"graph":[] , "nodes":[] , "edges":[],"hit":[] , "page":[] , "sim":[] }

for g_idx , d_path in enumerate(data_path[0:5]):
    record["graph"].append(g_idx+1)

    graph_data = load_graph_data(d_path)
    graph = Graph(graph_data)
    
    record["nodes"].append(graph.n)
    record["edges"].append(len(graph_data))

    for algo in ["hit","page","sim"]:

        s = time.time()
        if algo == "hit":
            _,_ =  graph.hits_algorithm(iter=ITER)
        elif algo == "page":
            _ = graph.pagerank_algorithm(iter=ITER,d= DAMPING_FACTOR_D)
        else:
            _ = graph.simrank_algorithm(iter=ITER,C = DECAY_FACTOR_C)

        run_time = time.time() -s
        record[algo].append(run_time)

In [14]:
import pandas as pd
run_time_df = pd.DataFrame(data=record)
# run time df
run_time_df

,graph,nodes,edges,hit,page,sim
0,1,6,5,0.004631,0.001701,0.004532
1,2,5,5,0.002604,0.000985,0.003860
2,3,4,6,0.002208,0.000897,0.002971
3,4,7,18,0.003498,0.001604,0.015365
4,5,469,1102,0.195385,0.076336,72.987632
